# Problème des seaux d'eau

**Comment obtenir un volume de 4L d'eau en utilisant un seau de 5L et un seau de 3L ?**

Pour obtenir exactement 4 litres d'eau en utilisant un seau de 5 litres, un seau de 3 litres et un seau de 4 litres, voici une méthode :

1. Remplissez le seau de 5 litres complètement (vous avez maintenant 5L dans le seau de 5L).
2. Versez de l'eau du seau de 5 litres dans le seau de 3 litres jusqu'à ce que le seau de 3 litres soit plein. Cela vous laissera avec 2 litres dans le seau de 5 litres (5L - 3L = 2L).
3. Videz le seau de 3 litres.
4. Versez les 2 litres restants du seau de 5 litres dans le seau de 3 litres.
5. Remplissez à nouveau le seau de 5 litres complètement (vous avez maintenant 5L dans le seau de 5L).
6. Versez de l'eau du seau de 5 litres dans le seau de 3 litres jusqu'à ce que le seau de 3 litres soit plein. Le seau de 3 litres a déjà 2 litres, donc vous pouvez y ajouter 1 litre supplémentaire (3L - 2L = 1L). Cela signifie que vous devrez retirer 1 litre du seau de 5 litres. Il vous restera exactement 4 litres d'eau dans le seau de 5 litres.

Ainsi, vous avez maintenant 4 litres d'eau dans le seau de 5 litres.

In [1]:
from typing import Type, Generic, TypeVar, Self, List, Tuple, Callable

## Seau d'eau

Un seau d'eau est défini par le volume d'eau contenu et sa capacité maximale. 

In [2]:
class Bucket():

    def __init__(self, name: str, max_capacity: float, volume: float = 0):
        self.name = name
        self.max_capacity = max_capacity
        self.volume = volume

    def __str__(self) -> str:
        return f"Bucket {self.name} contains {self.volume}L over {self.max_capacity}L"
    
    @property
    def remaining_volume(self) -> float:
        return self.max_capacity - self.volume

    @property
    def is_empty(self) -> bool:
        return self.volume == 0
    
    @property
    def is_full(self) -> bool:
        return self.volume == self.max_capacity
    
    @property
    def state(self) -> Tuple[float, float]:
        return self.volume, self.max_capacity

    def empty(self) -> 'Bucket':
        return Bucket(
            name=self.name,
            max_capacity=self.max_capacity,
            volume=0
        )
    
    def fill(self) -> 'Bucket':
        return Bucket(
            name=self.name,
            max_capacity=self.max_capacity,
            volume=self.max_capacity
        )
    
    def pour(self, bucket: 'Bucket') -> Tuple['Bucket', 'Bucket', float]:
        assert self.volume > 0, "Bucket is empty !"
        assert bucket.volume < bucket.max_capacity, "Destination bucket is already full !"

        divided_volume = min(self.volume, bucket.remaining_volume)

        return (Bucket(
            name=self.name,
            max_capacity=self.max_capacity,
            volume=self.volume - divided_volume
        ), Bucket(
            name=bucket.name,
            max_capacity=bucket.max_capacity,
            volume=bucket.volume + divided_volume
        ),
        divided_volume)


def test_class_bucket():

    A = Bucket('A', 5)
    B = Bucket('B', 3)

    A = A.fill()
    A, B, _ = A.pour(B)
    B = B.empty()
    A, B, _ = A.pour(B)
    A = A.fill()
    A, B, vol = A.pour(B)

    assert A.volume == 4
    assert B.is_full
    assert vol == 1

test_class_bucket()

## Etat du problème

Chaque étét du problème est défini par un ensemble de seaux qui contienent chacun un volume d'eau pour une capacité maximale.

In [3]:
class State:

    def __init__(self, buckets: List[Bucket], transition: str = '') -> None:
        self.buckets = buckets
        self.transition = transition

    def __eq__(self, other: Self) -> bool:
        
        assert isinstance(other, self.__class__), "Not implemented !"

        if len(self.buckets) != len(other.buckets):
            return False
        
        n_founded = 0

        for bucket in self.buckets:
            
            founded = 0            
            for other_bucket in other.buckets:
                if bucket.state == other_bucket.state:
                    founded = 1

            n_founded += founded

        return n_founded == len(self.buckets)
    
    def __ne__(self, other: Self) -> bool:
        
        assert isinstance(other, self.__class__), "Not implemented !"
        return not self.__eq__(other)


def test_class_state():

    b1 = Bucket('A', 5, 2)
    b2 = Bucket('B', 3, 1)
    b3 = Bucket('C', 7, 0)
    b4 = Bucket('D', 3, 1)

    s1 = State([b1, b2])
    s2 = State([b2, b3])
    s3 = State([b4, b1])
    s4 = State([b1, b2, b3])

    assert s1 != s2
    assert s2 != s3
    assert s1 == s3
    assert s1 != s4

## Arbre

Le modèle est construit sur un arbre d'états de seaux d'eau.

In [4]:
T = TypeVar("T")

class Node(Generic[T]):

    def __init__(self, parent_node: 'Node[T] | None', value: T):
        
        self.parent_node = parent_node
        self.value = value        
        self.children: List[Node[T]] | None = None

    def append_child(self, node: 'Node[T]') -> None:
        
        if self.children is None:
            self.children = []

        self.children.append(node)


class Tree(Generic[T]):

    def __init__(self, root_node: Node[T]):
        
        self.root_node = root_node
        
    def navigate_down(self, node: Node[T], action: Callable[[Node[T]], None]) -> None:
        self.recursive_down(node, node, action, 0)

    def recursive_down(self, node: Node[T], start_node: Node[T], action: Callable[[Node[T]], None], depth: int, depth_limit = 100) -> None:

        assert depth <= depth_limit, "Too many recursivity"
        
        action(node)

        if node.children is not None:
            for child in node.children:
                assert child != start_node, "Loop detected"                
                self.recursive_down(child, start_node, action, depth + 1)

    def navigate_up(self, node: Node[T], action: Callable[[Node[T]], None], max_depth = 100) -> None:
        self.recursive_up(node, node, action, 0, max_depth)

    def recursive_up(self, node: Node[T], start_node: Node[T], action: Callable[[Node[T]], None], depth: int, depth_limit = 100) -> None:

        assert depth <= depth_limit, "Too many recursivity"
        
        action(node)

        if node.parent_node is not None:
            assert node.parent_node != start_node, "Loop detected"
            self.recursive_up(node.parent_node, start_node, action, depth)


def test_class_tree():

    # Tree of interger nodes
    #
    #   3
    #  /|\
    # 5 0 2
    # |
    # 4

    root_node = Node(
        parent_node=None,
        value=3
    )

    for v in [5, 0, 2]:
        root_node.append_child(Node(
            parent_node=root_node,
            value=v
        ))
    
    root_node.children[0].append_child(Node( # type: ignore
        parent_node=root_node.children[0], # type: ignore
        value=4
    ))

    t = Tree(root_node=root_node)

    s = 0
    def calculate_sum(node: Node[int]):
        nonlocal s
        s += node.value
    
    t.navigate_down(t.root_node, calculate_sum)
    assert s == 14

    s = 0
    
    t.navigate_up(t.root_node.children[0].children[0], calculate_sum) # type: ignore
    assert s == 12

test_class_tree()

## Problème principal

In [5]:
class Problem:

    def __init__(self, initial_state: State, objective_volumes: List[float], can_drop = True, can_fill = True) -> None:
        self.tree = Tree(
            root_node=Node(
                parent_node=None,
                value=initial_state
            )
        )
        self.objective_states = State(
            buckets=list(map(lambda o: Bucket(
                name=o[0].name,
                max_capacity=o[0].max_capacity,
                volume=o[1]
            ), zip(initial_state.buckets, objective_volumes)))
        )
        self.reached_objective_nodes: List[Tuple[Node[State], int]] = []
        self.can_drop = can_drop
        self.can_fill = can_fill

    def solve(self, max_depth: int = 10) -> None:
        
        self.build_recursively(
            state_node=self.tree.root_node,
            depth=0,
            max_depth=max_depth
        )

        if len(self.reached_objective_nodes) > 0:

            print(f'{len(self.reached_objective_nodes)} solutions found !')

            min_depth = max_depth
            best_node_idx = 0
            for idx, (_, depth) in enumerate(self.reached_objective_nodes):
                if depth < min_depth:
                    min_depth = depth
                    best_node_idx = idx
            
            best_node = self.reached_objective_nodes[best_node_idx][0]

            print(f'Best solution in {min_depth} steps :')

            steps: List[str] = []

            def get_transition(node: Node[State]):
                nonlocal steps
                steps.append(f'{node.value.transition}')

            self.tree.navigate_up(
                node=best_node,
                action=get_transition
            )

            for idx, step in enumerate(reversed(steps)):
                print(f'{idx}. {step}')

        else:
            print('No solution found !')


    def add_child_if_state_not_already_exists(self, state: State, parent_node: Node[State]) -> None:
        already_exists = False

        def check_if_state_already_exists(node: Node[State]):
            nonlocal already_exists
            if node.value == state:
                already_exists = True

        self.tree.navigate_up(node=parent_node, action=check_if_state_already_exists)

        if not already_exists:
            parent_node.append_child(Node(
                parent_node=parent_node,
                value=state
            ))


    def build_recursively(self, state_node: Node[State], depth: int, max_depth: int = 100) -> None:

        state = state_node.value # alias

        # Build all the possible operations on each bucket
        for i, bucket in enumerate(state.buckets):

            # Empty
            if not bucket.is_empty and self.can_drop:
                self.add_child_if_state_not_already_exists(
                    parent_node=state_node,
                    state=self.empty_bucket(
                        state=state,
                        idx=i
                    )
                )

            # Fill
            if not bucket.is_full and self.can_fill:
                self.add_child_if_state_not_already_exists(
                    parent_node=state_node,
                    state=self.fill_bucket(
                        state=state,
                        idx=i
                    )
                )

            # Divide in another bucket
            if not bucket.is_empty:
                for j, other in enumerate(state.buckets):
                    if i != j and not other.is_full:
                        self.add_child_if_state_not_already_exists(
                            parent_node=state_node,
                            state=self.pour_bucket(
                                state=state,
                                idx=i,
                                jdx=j
                            )
                        )

        # For all new states
        if state_node.children is not None and depth + 1 <= max_depth:
            for child_node in state_node.children:

                # Check for reached objective
                if child_node.value == self.objective_states:
                    self.reached_objective_nodes.append((child_node, depth + 1))

                # If not, continue
                else:
                    self.build_recursively(child_node, depth + 1, max_depth)



    def empty_bucket(self, state: State, idx: int) -> State:

        buckets = []
        for i, bucket in enumerate(state.buckets):
            if i == idx:
                buckets.append(bucket.empty())
            else:
                buckets.append(bucket)

        return State(
            buckets=buckets,
            transition=f'Empty bucket {state.buckets[idx].name}.'
        )
    

    def fill_bucket(self, state: State, idx: int) -> State:

        buckets = []
        for i, bucket in enumerate(state.buckets):
            if i == idx:
                buckets.append(bucket.fill())
            else:
                buckets.append(bucket)

        return State(
            buckets=buckets,
            transition=f'Fill bucket {state.buckets[idx].name}.'
        )
    
    def pour_bucket(self, state: State, idx: int, jdx: int) -> State:

        buckets = []

        a, b, vol = state.buckets[idx].pour(state.buckets[jdx])

        for i, bucket in enumerate(state.buckets):
            if i == idx:
                buckets.append(a)
            elif i == jdx:
                buckets.append(b)
            else:
                buckets.append(bucket)

        return State(
            buckets=buckets,
            transition=f'Pour {vol}L from bucket {a.name} in bucket {b.name}. Bucket {a.name} contains now {a.volume}L and bucket {b.name} contains now {b.volume}L.'
        )
    

def test_problem():
    
    b = Bucket(
        name='A',
        max_capacity=5,
        volume=0
    )

    s = State(
        buckets=[
            b.fill(),
            b
        ],
        transition=''
    )

    p = Problem(
        initial_state=s,
        objective_volumes=[]
    )

    assert s.buckets[0].is_full

    s = p.empty_bucket(s, 0)

    assert s.buckets[0].is_empty

    s = p.fill_bucket(s, 0)

    assert s.buckets[0].is_full

    s = p.pour_bucket(s, 0, 1)

    assert s.buckets[0].is_empty
    assert s.buckets[1].is_full


test_problem()

## Résolution du problème initial

In [6]:
Problem(
    initial_state=State(
        buckets=[
            Bucket(
                name='A',
                max_capacity=5,
                volume=0
            ),
            Bucket(
                name='B',
                max_capacity=3,
                volume=0
            )
        ],
        transition='Initial state'
    ),
    objective_volumes=[4, 0]
).solve()

6 solutions found !
Best solution in 7 steps :
0. Initial state
1. Fill bucket A.
2. Pour 3L from bucket A in bucket B. Bucket A contains now 2L and bucket B contains now 3L.
3. Empty bucket B.
4. Pour 2L from bucket A in bucket B. Bucket A contains now 0L and bucket B contains now 2L.
5. Fill bucket A.
6. Pour 1L from bucket A in bucket B. Bucket A contains now 4L and bucket B contains now 3L.
7. Empty bucket B.


## Variante

In [9]:
Problem(
    initial_state=State(
        buckets=[
            Bucket(
                name='A',
                max_capacity=8,
                volume=8
            ),
            Bucket(
                name='B',
                max_capacity=5,
                volume=0
            ),
            Bucket(
                name='B',
                max_capacity=3,
                volume=0
            )
        ],
        transition='Initial state'
    ),
    objective_volumes=[4, 4, 0],
    can_fill=False,
    can_drop=False
).solve()

6 solutions found !
Best solution in 7 steps :
0. Initial state
1. Pour 5L from bucket A in bucket B. Bucket A contains now 3L and bucket B contains now 5L.
2. Pour 3L from bucket B in bucket B. Bucket B contains now 2L and bucket B contains now 3L.
3. Pour 3L from bucket B in bucket A. Bucket B contains now 0L and bucket A contains now 6L.
4. Pour 2L from bucket B in bucket B. Bucket B contains now 0L and bucket B contains now 2L.
5. Pour 5L from bucket A in bucket B. Bucket A contains now 1L and bucket B contains now 5L.
6. Pour 1L from bucket B in bucket B. Bucket B contains now 4L and bucket B contains now 3L.
7. Pour 3L from bucket B in bucket A. Bucket B contains now 0L and bucket A contains now 4L.


In [10]:
Problem(
    initial_state=State(
        buckets=[
            Bucket(
                name='A',
                max_capacity=10,
                volume=10
            ),
            Bucket(
                name='B',
                max_capacity=7,
                volume=0
            ),
            Bucket(
                name='C',
                max_capacity=3,
                volume=0
            )
        ],
        transition='Initial state'
    ),
    objective_volumes=[5, 5, 0],
    can_fill=False,
    can_drop=False
).solve()

2 solutions found !
Best solution in 9 steps :
0. Initial state
1. Pour 7L from bucket A in bucket B. Bucket A contains now 3L and bucket B contains now 7L.
2. Pour 3L from bucket B in bucket C. Bucket B contains now 4L and bucket C contains now 3L.
3. Pour 3L from bucket C in bucket A. Bucket C contains now 0L and bucket A contains now 6L.
4. Pour 3L from bucket B in bucket C. Bucket B contains now 1L and bucket C contains now 3L.
5. Pour 3L from bucket C in bucket A. Bucket C contains now 0L and bucket A contains now 9L.
6. Pour 1L from bucket B in bucket C. Bucket B contains now 0L and bucket C contains now 1L.
7. Pour 7L from bucket A in bucket B. Bucket A contains now 2L and bucket B contains now 7L.
8. Pour 2L from bucket B in bucket C. Bucket B contains now 5L and bucket C contains now 3L.
9. Pour 3L from bucket C in bucket A. Bucket C contains now 0L and bucket A contains now 5L.
